# Initialization

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize simulator

In [44]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 9
ROUNDS = 9
OTHER_DATE = '2023-10-27'
_RESETS = False
LOGICAL = 0 # NOT NEEDED FOR EXTREME IQ BCS HARDCODED 0

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

# Get IQ data

In [45]:
SHOTS = int(2e5)
NOISE_LIST = [3e-2, 0.8e-2, 1e-2, 3e-2] # [two-qubit-fidelity, reset error, measurement error, idle error]
# NOISE_LIST = None
P_AMBIG = 0.1

IQ_data = simulator.generate_IQ(SHOTS, noise_list=NOISE_LIST)
# IQ_data_extreme = simulator.generate_extreme_IQ(SHOTS, P_AMBIG, noise_list=NOISE_LIST)

# Decode using predecoder

In [46]:
import pymatching
import stim
import cpp_soft_info
import numpy as np

model = simulator.stim_circ.detector_error_model(decompose_errors=False)
matching = pymatching.Matching.from_detector_error_model(model)

In [41]:
_DETAILED = False

result_predecoder = cpp_soft_info.decode_time_nn_predecode_grid(model, IQ_data, ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, threshold=0.15, _ntnn_edges = not _RESETS)


print(f"num_errors: {result_predecoder.decode_result.num_errors}")
print(f"mean num removed edges: {np.mean(result_predecoder.nb_rm_edges)}")
print(f"num removed edges: {np.sum(result_predecoder.nb_rm_edges)}")

num_errors: 5372
mean num removed edges: 0.051935
num removed edges: 10387


In [30]:
for threshold in np.linspace(0, 0.4, 20):
    result_predecoder = cpp_soft_info.decode_time_nn_predecode_grid(model, IQ_data[:], ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                               simulator.processed_scaler_dict, _detailed=_DETAILED, threshold=threshold, _ntnn_edges = not _RESETS)
    print(f"threshold: {threshold}, num_errors: {result_predecoder.decode_result.num_errors}, mean num removed edges: {np.mean(result_predecoder.nb_rm_edges)}, num removed edges: {np.sum(result_predecoder.nb_rm_edges)}")

threshold: 0.0, num_errors: 520, mean num removed edges: 0.5321, num removed edges: 5321
threshold: 0.021052631578947368, num_errors: 268, mean num removed edges: 0.1155, num removed edges: 1155
threshold: 0.042105263157894736, num_errors: 264, mean num removed edges: 0.0802, num removed edges: 802
threshold: 0.06315789473684211, num_errors: 262, mean num removed edges: 0.0709, num removed edges: 709
threshold: 0.08421052631578947, num_errors: 261, mean num removed edges: 0.0622, num removed edges: 622
threshold: 0.10526315789473684, num_errors: 262, mean num removed edges: 0.0571, num removed edges: 571
threshold: 0.12631578947368421, num_errors: 263, mean num removed edges: 0.0531, num removed edges: 531
threshold: 0.14736842105263157, num_errors: 263, mean num removed edges: 0.0504, num removed edges: 504
threshold: 0.16842105263157894, num_errors: 265, mean num removed edges: 0.0477, num removed edges: 477
threshold: 0.18947368421052632, num_errors: 265, mean num removed edges: 0.0

In [42]:
# Soft Grid info

result = cpp_soft_info.decode_IQ_fast(model, IQ_data,
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict,
                                           simulator.processed_scaler_dict, _detailed=_DETAILED, nb_intervals=-1)

print(f"num_errors: {result.num_errors}")

num_errors: 4890


In [47]:
# Hard decoder
# p_meas = 0.02
p_meas = -1

result_informed = cpp_soft_info.decode_IQ_shots_flat_informed(model, IQ_data[:],
                                           ROUNDS, int(LOGICAL), _RESETS, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict,
                                           p_data = -1, p_mixed = -1, p_meas = p_meas, common_measure=-1, _detailed=_DETAILED, _ntnn_edges = not _RESETS)

print(f"num_errors: {result_informed.num_errors}")

num_errors: 2333
